In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/3/22,6/4/22,6/5/22,6/6/22,6/7/22,6/8/22,6/9/22,6/10/22,6/11/22,6/12/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,180584,180615,180615,180688,180741,180784,180864,180864,180864,180864
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,276310,276342,276401,276415,276468,276518,276583,276638,276690,276731
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,265889,265889,265897,265900,265904,265909,265920,265925,265925,265927
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,43067,43067,43067,43067,43067,43224,43224,43224,43224,43224
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,99761,99761,99761,99761,99761,99761,99761,99761,99761,99761


In [5]:
del df["Lat"]
del df["Long"]

In [6]:
countries_df = df[["Province/State","Country/Region"]]
countries_df.head()

,Province/State,Country/Region
0,NaN,Afghanistan
1,NaN,Albania
2,NaN,Algeria
3,NaN,Andorra
4,NaN,Angola


In [7]:
days_df = df.copy()
del days_df["Province/State"]
del days_df["Country/Region"]

In [8]:
days_df = days_df.stack().reset_index()
days_df.columns=["country_id", "date", "count"]
days_df.head()

,country_id,date,count
0,0,1/22/20,0
1,0,1/23/20,0
2,0,1/24/20,0
3,0,1/25/20,0
4,0,1/26/20,0


In [11]:
df = pd.merge(countries_df, days_df, left_index=True, right_on="country_id")
del df["country_id"]
df["date"] = pd.to_datetime(df["date"])
df.head()

,Province/State,Country/Region,date,count
0,NaN,Afghanistan,2020-01-22,0
1,NaN,Afghanistan,2020-01-23,0
2,NaN,Afghanistan,2020-01-24,0
3,NaN,Afghanistan,2020-01-25,0
4,NaN,Afghanistan,2020-01-26,0


In [12]:
global_df = df.groupby(["date"],as_index=False).sum("count")

In [13]:
global_df.tail()

,date,count
868,2022-06-08,533663415
869,2022-06-09,534211200
870,2022-06-10,534787230
871,2022-06-11,535063602
872,2022-06-12,535259297


In [14]:
latest_date_str = global_df.date.max().strftime("%Y-%m-%d")

In [15]:
import matplotlib.pyplot as plt
%matplotlib notebook
#%matplotlib inline

In [16]:
df["Country/Region"].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana',
       'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burma', 'Burundi',
       'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Comoros', 'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark',
       'Diamond Princess', 'Djibouti', 'Dominica', 'Dominican Republic',
       'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea',
       'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France',
       'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece',
       'Grenada', 'Gua

In [17]:
#plt.figure(figsize=(9.5,5))
#plt.bar(global_df["date"],global_df["count"]/1000,label="positive")
#plt.legend()

In [18]:
trans = pd.DataFrame()
trans["date"] = global_df.iloc[1:]["date"]
trans["count1"] = global_df["count"].iloc[1:]
trans = trans.reset_index().drop(columns="index")
trans["count"] = trans["count1"] - global_df["count"].iloc[0:]
del trans["count1"]
trans["move_ave7"] = trans["count"].rolling(7).mean()


In [19]:
trans.tail()

,date,count,move_ave7
867,2022-06-08,710960.0,479128.428571
868,2022-06-09,547785.0,482945.285714
869,2022-06-10,576030.0,488228.571429
870,2022-06-11,276372.0,492136.285714
871,2022-06-12,195695.0,483623.428571


In [21]:
fig=plt.figure(figsize=(9.5,8))
plt.subplot(2,1,1) # (rows, columns, panel number)
plt.bar(trans["date"],trans["count"],label="New cases")
plt.plot_date(trans["date"],trans["move_ave7"],label="rolling 7-day average", fmt="-", color="orange")
plt.title("Daily new confirmed COVID-19 cases in the world({})".format(latest_date_str))
plt.ylabel("people")
plt.grid(axis="y")
plt.legend()

plt.subplot(2,1,2) # (rows, columns, panel number)
plt.title("Daily new confirmed COVID-19 cases in the world(after 2021 Mar)({})".format(latest_date_str))
chart_data = trans[trans.date >= "2021-03-01"]
plt.bar(chart_data["date"],chart_data["count"],label="New cases")
plt.plot_date(chart_data["date"],chart_data["move_ave7"],label="rolling 7-day average", fmt="-", color="orange")
plt.grid(axis="y")
plt.legend()
#plt.legend(bbox_to_anchor=(0, 1),loc='upper left', fontsize=9)
fig.text(0,0
         ,"*Data reference from:COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University",
        fontsize=7,ha="left",va="bottom")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [22]:
fig.savefig("out/covid-global-transition.png")

## 国別

In [23]:
df.tail()

,Province/State,Country/Region,date,count
248800,NaN,Zimbabwe,2022-06-08,253779
248801,NaN,Zimbabwe,2022-06-09,253779
248802,NaN,Zimbabwe,2022-06-10,254031
248803,NaN,Zimbabwe,2022-06-11,254031
248804,NaN,Zimbabwe,2022-06-12,254098


### 中国の合計行を追加する

In [24]:
china_sum = df[df["Country/Region"]=="China"].groupby("date", as_index=False).sum("count")
china_sum

,date,count
0,2020-01-22,548
1,2020-01-23,643
2,2020-01-24,920
3,2020-01-25,1406
4,2020-01-26,2075
...,...,...
868,2022-06-08,2101560
869,2022-06-09,2102379
870,2022-06-10,2103227
871,2022-06-11,2104341


In [25]:
import numpy as np
china_sum.insert(0,"Country/Region","China")
china_sum.insert(0,"Province/State",np.nan)
china_sum

,Province/State,Country/Region,date,count
0,NaN,China,2020-01-22,548
1,NaN,China,2020-01-23,643
2,NaN,China,2020-01-24,920
3,NaN,China,2020-01-25,1406
4,NaN,China,2020-01-26,2075
...,...,...,...,...
868,NaN,China,2022-06-08,2101560
869,NaN,China,2022-06-09,2102379
870,NaN,China,2022-06-10,2103227
871,NaN,China,2022-06-11,2104341


In [26]:
df = pd.concat([df,china_sum])

### 国別新規陽性者数

In [27]:
trans = df.copy()
trans["inc_count"] = trans["count"].diff()
# 最小の日付は一律ゼロにする
trans.loc[trans["date"]==trans["date"].min(),"inc_count"] = 0

In [28]:
## 異常値？の訂正
import numpy as np
trans.loc[(trans["Country/Region"]=="France") & (trans.inc_count < -300000),"inc_count"]=np.nan

#### 移動平均

In [29]:
# "Province/State"と"Country/Region"の結合列を作ってユニークにする
trans["country_key"]=trans["Province/State"].fillna("NA")+"-"+trans["Country/Region"]
if trans["country_key"].isna().any():
    raise Exception("include NaN")
for k in trans.country_key.unique():
    print(k,end=",")
    trans.loc[trans.country_key==k,"cnt_move_ave7"] = trans.loc[trans.country_key==k,"count"].rolling(7).mean()
    trans.loc[trans.country_key==k,"inc_move_ave7"] = trans.loc[trans.country_key==k,"inc_count"].rolling(7).mean()

pd.concat([trans[trans.country_key=="NA-Afghanistan"].tail(3),trans[trans.country_key=="NA-Albania"].head(3)])

NA-Afghanistan,NA-Albania,NA-Algeria,NA-Andorra,NA-Angola,NA-Antarctica,NA-Antigua and Barbuda,NA-Argentina,NA-Armenia,Australian Capital Territory-Australia,New South Wales-Australia,Northern Territory-Australia,Queensland-Australia,South Australia-Australia,Tasmania-Australia,Victoria-Australia,Western Australia-Australia,NA-Austria,NA-Azerbaijan,NA-Bahamas,NA-Bahrain,NA-Bangladesh,NA-Barbados,NA-Belarus,NA-Belgium,NA-Belize,NA-Benin,NA-Bhutan,NA-Bolivia,NA-Bosnia and Herzegovina,NA-Botswana,NA-Brazil,NA-Brunei,NA-Bulgaria,NA-Burkina Faso,NA-Burma,NA-Burundi,NA-Cabo Verde,NA-Cambodia,NA-Cameroon,Alberta-Canada,British Columbia-Canada,Diamond Princess-Canada,Grand Princess-Canada,Manitoba-Canada,New Brunswick-Canada,Newfoundland and Labrador-Canada,Northwest Territories-Canada,Nova Scotia-Canada,Nunavut-Canada,Ontario-Canada,Prince Edward Island-Canada,Quebec-Canada,Repatriated Travellers-Canada,Saskatchewan-Canada,Yukon-Canada,NA-Central African Republic,NA-Chad,NA-Chile,Anhui-China,

,Province/State,Country/Region,date,count,inc_count,country_key,cnt_move_ave7,inc_move_ave7
870,NaN,Afghanistan,2022-06-10,180864,0.0,NA-Afghanistan,180738.714286,40.000000
871,NaN,Afghanistan,2022-06-11,180864,0.0,NA-Afghanistan,180774.285714,35.571429
872,NaN,Afghanistan,2022-06-12,180864,0.0,NA-Afghanistan,180809.857143,35.571429
873,NaN,Albania,2020-01-22,0,0.0,NA-Albania,NaN,NaN
874,NaN,Albania,2020-01-23,0,0.0,NA-Albania,NaN,NaN
875,NaN,Albania,2020-01-24,0,0.0,NA-Albania,NaN,NaN


In [30]:
trans["Country/Region"].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana',
       'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burma', 'Burundi',
       'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Comoros', 'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark',
       'Diamond Princess', 'Djibouti', 'Dominica', 'Dominican Republic',
       'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea',
       'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France',
       'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece',
       'Grenada', 'Gua

## 人口当たり

In [31]:
pop = pd.read_excel("world-data/WPP2019_POP_F01_1_TOTAL_POPULATION_BOTH_SEXES.xlsx",header=16)
pop_latest = pop.loc[pop["Type"]=="Country/Area"][[pop.columns[2],pop.columns[-1]]]
pop_latest.columns = ["un_country","population"]
pop_latest.head(1)

,un_country,population
26,Burundi,11890.781


### 名称変換

In [32]:
alternates = pd.DataFrame([
    ("US","United States of America"),
    ("Russia","Russian Federation"),
    ("Iran","Iran (Islamic Republic of)"),
    ("Bolivia","Bolivia (Plurinational State of)"),
    ("Moldova","Republic of Moldova"),
    ("Venezuela","Venezuela (Bolivarian Republic of)"),
    ("Korea, South","Republic of Korea"),
    ("Congo (Kinshasa)","Democratic Republic of the Congo"),
    ("Congo (Brazzaville)","Congo"),
    ("Vietnam","Viet Nam"),
    ("Laos","Lao People's Democratic Republic"),
    ("Taiwan*","China, Taiwan Province of China"),
    ("Tanzania","United Republic of Tanzania"),
    ("Brunei","Brunei Darussalam"),
    ("Micronesia","Micronesia (Fed. States of)"),
    ("Syria","Syrian Arab Republic"),
], columns = ["covid_country","un_country"])

In [33]:
alternates.head(2)

,covid_country,un_country
0,US,United States of America
1,Russia,Russian Federation


In [34]:
trans_pop = pd.merge(trans, alternates, left_on="Country/Region", right_on="covid_country", how="left")
del trans_pop["covid_country"]
trans_pop.rename(columns={"Country/Region":"country"},inplace=True)

In [35]:
trans_pop.head()

,Province/State,country,date,count,inc_count,country_key,cnt_move_ave7,inc_move_ave7,un_country
0,NaN,Afghanistan,2020-01-22,0,0.0,NA-Afghanistan,NaN,NaN,NaN
1,NaN,Afghanistan,2020-01-23,0,0.0,NA-Afghanistan,NaN,NaN,NaN
2,NaN,Afghanistan,2020-01-24,0,0.0,NA-Afghanistan,NaN,NaN,NaN
3,NaN,Afghanistan,2020-01-25,0,0.0,NA-Afghanistan,NaN,NaN,NaN
4,NaN,Afghanistan,2020-01-26,0,0.0,NA-Afghanistan,NaN,NaN,NaN


In [36]:
trans_pop.isnull().any(axis=0)

Province/State     True
country           False
date              False
count             False
inc_count          True
country_key       False
cnt_move_ave7      True
inc_move_ave7      True
un_country         True
dtype: bool

In [37]:
trans_pop["un_country"].fillna(trans_pop["country"],inplace=True)

In [38]:
trans_pop.isnull().any(axis=0)

Province/State     True
country           False
date              False
count             False
inc_count          True
country_key       False
cnt_move_ave7      True
inc_move_ave7      True
un_country        False
dtype: bool

In [39]:
trans_pop = pd.merge(trans_pop, pop_latest, on="un_country", how="left")

In [40]:
trans_pop.head()

,Province/State,country,date,count,inc_count,country_key,cnt_move_ave7,inc_move_ave7,un_country,population
0,NaN,Afghanistan,2020-01-22,0,0.0,NA-Afghanistan,NaN,NaN,Afghanistan,38928.341
1,NaN,Afghanistan,2020-01-23,0,0.0,NA-Afghanistan,NaN,NaN,Afghanistan,38928.341
2,NaN,Afghanistan,2020-01-24,0,0.0,NA-Afghanistan,NaN,NaN,Afghanistan,38928.341
3,NaN,Afghanistan,2020-01-25,0,0.0,NA-Afghanistan,NaN,NaN,Afghanistan,38928.341
4,NaN,Afghanistan,2020-01-26,0,0.0,NA-Afghanistan,NaN,NaN,Afghanistan,38928.341


In [41]:
trans_pop["count_per_mil_pop"] = trans_pop["count"] / (trans_pop["population"] / 1000)
trans_pop["inc_count_per_mil_pop"] = trans_pop["inc_count"] / (trans_pop["population"] / 1000)
trans_pop["cnt_move_ave7_milpop"] = trans_pop["cnt_move_ave7"] / (trans_pop["population"] / 1000)
trans_pop["inc_move_ave7_milpop"] = trans_pop["inc_move_ave7"] / (trans_pop["population"] / 1000)

In [42]:
pd.concat([trans_pop.head(),trans_pop.tail()])

,Province/State,country,date,count,inc_count,country_key,cnt_move_ave7,inc_move_ave7,un_country,population,count_per_mil_pop,inc_count_per_mil_pop,cnt_move_ave7_milpop,inc_move_ave7_milpop
0,NaN,Afghanistan,2020-01-22,0,0.0,NA-Afghanistan,NaN,NaN,Afghanistan,38928.341,0.0,0.0,NaN,NaN
1,NaN,Afghanistan,2020-01-23,0,0.0,NA-Afghanistan,NaN,NaN,Afghanistan,38928.341,0.0,0.0,NaN,NaN
2,NaN,Afghanistan,2020-01-24,0,0.0,NA-Afghanistan,NaN,NaN,Afghanistan,38928.341,0.0,0.0,NaN,NaN
3,NaN,Afghanistan,2020-01-25,0,0.0,NA-Afghanistan,NaN,NaN,Afghanistan,38928.341,0.0,0.0,NaN,NaN
4,NaN,Afghanistan,2020-01-26,0,0.0,NA-Afghanistan,NaN,NaN,Afghanistan,38928.341,0.0,0.0,NaN,NaN
249673,NaN,China,2022-06-08,2101560,511.0,NA-China,2.099806e+06,611.142857,China,1439323.774,1460.102333,0.355028,1458.883705,0.424604
249674,NaN,China,2022-06-09,2102379,819.0,NA-China,2.100442e+06,636.285714,China,1439323.774,1460.67135,0.569017,1459.325778,0.442073
249675,NaN,China,2022-06-10,2103227,848.0,NA-China,2.101104e+06,661.714286,China,1439323.774,1461.260516,0.589166,1459.785517,0.45974
249676,NaN,China,2022-06-11,2104341,1114.0,NA-China,2.101861e+06,757.142857,China,1439323.774,1462.03449,0.773975,1460.311558,0.526041
249677,NaN,China,2022-06-12,2105331,990.0,NA-China,2.102642e+06,780.857143,China,1439323.774,1462.722313,0.687823,1460.854075,0.542517


In [43]:
trans_pop.loc[trans_pop["country"]=="Japan"].tail()


,Province/State,country,date,count,inc_count,country_key,cnt_move_ave7,inc_move_ave7,un_country,population,count_per_mil_pop,inc_count_per_mil_pop,cnt_move_ave7_milpop,inc_move_ave7_milpop
137056,NaN,Japan,2022-06-08,8978571,18394.0,NA-Japan,8.931029e+06,16738.000000,Japan,126476.458,70990.057296,145.43418,70614.163513,132.340835
137057,NaN,Japan,2022-06-09,8995359,16788.0,NA-Japan,8.947296e+06,16266.857143,Japan,126476.458,71122.793461,132.736165,70742.779204,128.615692
137058,NaN,Japan,2022-06-10,9010943,15584.0,NA-Japan,8.963038e+06,15742.142857,Japan,126476.458,71246.010068,123.216607,70867.246185,124.466981
137059,NaN,Japan,2022-06-11,9026281,15338.0,NA-Japan,8.978366e+06,15328.142857,Japan,126476.458,71367.281649,121.271581,70988.439829,121.193644
137060,NaN,Japan,2022-06-12,9039662,13381.0,NA-Japan,8.993450e+06,15083.428571,Japan,126476.458,71473.079994,105.798345,71107.698613,119.258784


### 主要国の定義

In [44]:
"""
countries = pd.DataFrame({"country":[
    "Japan",
    "US",
    "India",
    "Sweden",
    "Brazil",
    "Israel",
    "United Kingdom",
    "France",
    "Russia",
]})
"""
countries = pd.DataFrame([
    ("Japan",""),
    ("US",""),
    ("India","20210415"),
    ("Sweden","20210511"),
    ("Brazil",""),
    ("Israel","20210315"),
    ("United Kingdom",""),
    ("France","20210425"),
    ("Russia",""),
],columns=["country","text_disp_date"])

In [45]:
#c_df = pd.merge(countries, trans_pop[trans_pop["Province/State"].isna()], left_on="country", right_on="Country/Region", how="inner")[["country","date","count","inc_count","move_ave7"]]
c_trans = pd.merge(countries, trans_pop[trans_pop["Province/State"].isna()], on="country", how="inner")


In [46]:
c_trans

,country,text_disp_date,Province/State,date,count,inc_count,country_key,cnt_move_ave7,inc_move_ave7,un_country,population,count_per_mil_pop,inc_count_per_mil_pop,cnt_move_ave7_milpop,inc_move_ave7_milpop
0,Japan,,NaN,2020-01-22,2,0.0,NA-Japan,NaN,NaN,Japan,126476.458,0.015813,0.0,NaN,NaN
1,Japan,,NaN,2020-01-23,2,0.0,NA-Japan,NaN,NaN,Japan,126476.458,0.015813,0.0,NaN,NaN
2,Japan,,NaN,2020-01-24,2,0.0,NA-Japan,NaN,NaN,Japan,126476.458,0.015813,0.0,NaN,NaN
3,Japan,,NaN,2020-01-25,2,0.0,NA-Japan,NaN,NaN,Japan,126476.458,0.015813,0.0,NaN,NaN
4,Japan,,NaN,2020-01-26,4,2.0,NA-Japan,NaN,NaN,Japan,126476.458,0.031626,0.015813,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7852,Russia,,NaN,2022-06-08,18094054,3595.0,NA-Russia,1.808350e+07,3725.285714,Russian Federation,145934.46,123987.535226,24.634346,123915.213133,25.527115
7853,Russia,,NaN,2022-06-09,18097767,3713.0,NA-Russia,1.808716e+07,3656.285714,Russian Federation,145934.46,124012.978155,25.442928,123940.267432,25.0543
7854,Russia,,NaN,2022-06-10,18101372,3605.0,NA-Russia,1.809074e+07,3583.714286,Russian Federation,145934.46,124037.681025,24.70287,123964.824444,24.557012
7855,Russia,,NaN,2022-06-11,18104944,3572.0,NA-Russia,1.809426e+07,3523.857143,Russian Federation,145934.46,124062.157766,24.476741,123988.971292,24.146847


In [47]:
dates = trans["date"].unique()
len(dates)

873

In [48]:
c_trans[c_trans["country"]=="Japan"].tail()

,country,text_disp_date,Province/State,date,count,inc_count,country_key,cnt_move_ave7,inc_move_ave7,un_country,population,count_per_mil_pop,inc_count_per_mil_pop,cnt_move_ave7_milpop,inc_move_ave7_milpop
868,Japan,,NaN,2022-06-08,8978571,18394.0,NA-Japan,8.931029e+06,16738.000000,Japan,126476.458,70990.057296,145.43418,70614.163513,132.340835
869,Japan,,NaN,2022-06-09,8995359,16788.0,NA-Japan,8.947296e+06,16266.857143,Japan,126476.458,71122.793461,132.736165,70742.779204,128.615692
870,Japan,,NaN,2022-06-10,9010943,15584.0,NA-Japan,8.963038e+06,15742.142857,Japan,126476.458,71246.010068,123.216607,70867.246185,124.466981
871,Japan,,NaN,2022-06-11,9026281,15338.0,NA-Japan,8.978366e+06,15328.142857,Japan,126476.458,71367.281649,121.271581,70988.439829,121.193644
872,Japan,,NaN,2022-06-12,9039662,13381.0,NA-Japan,8.993450e+06,15083.428571,Japan,126476.458,71473.079994,105.798345,71107.698613,119.258784


In [49]:
print(any(c_trans["date"].isnull()))
print(any(c_trans["date"].isna()))

False
False


In [50]:
c_trans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7857 entries, 0 to 7856
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   country                7857 non-null   object        
 1   text_disp_date         7857 non-null   object        
 2   Province/State         0 non-null      object        
 3   date                   7857 non-null   datetime64[ns]
 4   count                  7857 non-null   int64         
 5   inc_count              7856 non-null   float64       
 6   country_key            7857 non-null   object        
 7   cnt_move_ave7          7803 non-null   float64       
 8   inc_move_ave7          7796 non-null   float64       
 9   un_country             7857 non-null   object        
 10  population             7857 non-null   object        
 11  count_per_mil_pop      7857 non-null   object        
 12  inc_count_per_mil_pop  7856 non-null   object        
 13  cnt

### 東アジア

In [51]:
# 東Asia 主要国
e_asia_c = pd.DataFrame([
    ("Japan",""),
    ("China",""),
    ("Taiwan*",""),
    ("Korea, South","20210420"),
    ("Indonesia",""),
    ("Philippines",""),
    ("Vietnam",""),
],columns=["country","text_disp_date"])
e_asia_c

,country,text_disp_date
0,Japan,
1,China,
2,Taiwan*,
3,"Korea, South",20210420
4,Indonesia,
5,Philippines,
6,Vietnam,


In [52]:
e_a_trans = pd.merge(e_asia_c, trans_pop[trans_pop["Province/State"].isna()], on="country", how="inner")
e_a_trans

,country,text_disp_date,Province/State,date,count,inc_count,country_key,cnt_move_ave7,inc_move_ave7,un_country,population,count_per_mil_pop,inc_count_per_mil_pop,cnt_move_ave7_milpop,inc_move_ave7_milpop
0,Japan,,NaN,2020-01-22,2,0.0,NA-Japan,NaN,NaN,Japan,126476.458,0.015813,0.0,NaN,NaN
1,Japan,,NaN,2020-01-23,2,0.0,NA-Japan,NaN,NaN,Japan,126476.458,0.015813,0.0,NaN,NaN
2,Japan,,NaN,2020-01-24,2,0.0,NA-Japan,NaN,NaN,Japan,126476.458,0.015813,0.0,NaN,NaN
3,Japan,,NaN,2020-01-25,2,0.0,NA-Japan,NaN,NaN,Japan,126476.458,0.015813,0.0,NaN,NaN
4,Japan,,NaN,2020-01-26,4,2.0,NA-Japan,NaN,NaN,Japan,126476.458,0.031626,0.015813,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6106,Vietnam,,NaN,2022-06-08,10727918,913.0,NA-Vietnam,1.072530e+07,1070.285714,Viet Nam,97338.583,110212.391319,9.379631,110185.448545,10.995493
6107,Vietnam,,NaN,2022-06-09,10728720,802.0,NA-Vietnam,1.072616e+07,869.428571,Viet Nam,97338.583,110220.6306,8.239282,110194.380548,8.932004
6108,Vietnam,,NaN,2022-06-10,10729681,961.0,NA-Vietnam,1.072702e+07,858.285714,Viet Nam,97338.583,110230.503355,9.872755,110203.198077,8.817528
6109,Vietnam,,NaN,2022-06-11,10731244,1563.0,NA-Vietnam,1.072798e+07,955.714286,Viet Nam,97338.583,110246.560709,16.057353,110213.01653,9.818453


In [53]:
e_a_trans.count()

country                  6111
text_disp_date           6111
Province/State              0
date                     6111
count                    6111
inc_count                6111
country_key              6111
cnt_move_ave7            6069
inc_move_ave7            6069
un_country               6111
population               6111
count_per_mil_pop        6111
inc_count_per_mil_pop    6111
cnt_move_ave7_milpop     6069
inc_move_ave7_milpop     6069
dtype: int64

## チャート化

In [54]:
# 主要国　新規感染者数　全期間
plt.figure(figsize=(9.5,5))
plt.grid()
dates = c_trans["date"].unique()
for c in countries["country"]:
    print(c)
    plt.plot_date(dates,c_trans.loc[c_trans["country"]==c]["inc_count"],label=c, fmt="-")
plt.legend()
plt.tight_layout()

<IPython.core.display.Javascript object>

Japan
US
India
Sweden
Brazil
Israel
United Kingdom
France
Russia


In [55]:
# 主要国　新規感染者数７日間移動平均　全期間
plt.figure(figsize=(9.5,5))
plt.grid()
dates = c_trans["date"].unique()
for c in countries["country"]:
    print(c)
    plt.plot_date(dates,c_trans.loc[c_trans["country"]==c]["cnt_move_ave7"],label=c, fmt="-")
plt.legend()
plt.tight_layout()

<IPython.core.display.Javascript object>

Japan
US
India
Sweden
Brazil
Israel
United Kingdom
France
Russia


In [60]:
# 主要国　１００万人あたり感染者数累計 全期間　７日間移動平均
plt.figure(figsize=(9.5,5))
plt.grid()
plt.title("positives per million poplulasions transition(1w moving-ave)")
dates = c_trans["date"].unique()
for c in countries["country"]:
    plt.plot_date(dates,c_trans.loc[c_trans["country"]==c]["cnt_move_ave7_milpop"],label=c, fmt="-")
    
plt.legend()
plt.tight_layout()

<IPython.core.display.Javascript object>

In [61]:
# 主要国　１００万人あたり感染者数累計 1月以降　７日間移動平均
plt.figure(figsize=(9.5,5))
plt.grid()
plt.title("positives per million poplulasions transition(1w moving-ave)(21/1/1-)")
dates = c_trans.loc[c_trans["date"]>="2021-01-01"]["date"].unique()
for c in countries["country"]:
    plt.plot_date(dates,c_trans.loc[(c_trans["country"]==c) & (c_trans["date"]>="2021-01-01")]["cnt_move_ave7_milpop"],label=c, fmt="-")
    
plt.legend()
plt.tight_layout()

<IPython.core.display.Javascript object>

## 人口当たりの新規感染者数

In [62]:
# 主要国　１００万人あたり新規感染者数 全期間　７日間移動平均
"""
plt.figure(figsize=(9.5,5))
plt.grid()
plt.title("increase positives per million poplulasions transition(1w moving-ave)")
dates = c_trans["date"].unique()
for c in countries["country"]:
    plt.plot_date(dates,c_trans.loc[c_trans["country"]==c]["inc_move_ave7_milpop"],label=c, fmt="-")
    plt.text(dates.max(),c_trans.loc[(c_trans["country"]==c) & (c_trans["date"]==dates.max())]["inc_move_ave7_milpop"],c,fontsize=8)

plt.legend()
plt.tight_layout()
"""

'\nplt.figure(figsize=(9.5,5))\nplt.grid()\nplt.title("increase positives per million poplulasions transition(1w moving-ave)")\ndates = c_trans["date"].unique()\nfor c in countries["country"]:\n    plt.plot_date(dates,c_trans.loc[c_trans["country"]==c]["inc_move_ave7_milpop"],label=c, fmt="-")\n    plt.text(dates.max(),c_trans.loc[(c_trans["country"]==c) & (c_trans["date"]==dates.max())]["inc_move_ave7_milpop"],c,fontsize=8)\n\nplt.legend()\nplt.tight_layout()\n'

## 人口100万人当たりの新規感染者数（3月以降）

In [65]:
# 主要国　１００万人あたり感染者数累計 3月以降　７日間移動平均
fig=plt.figure(figsize=(9.5,10))
plt.subplot(2,1,1) # (rows, columns, panel number)
plt.grid()
plt.title("主要国の人口100万人当たりの新規感染者数７日間移動平均({}時点)".format(latest_date_str))
chart_data = c_trans[c_trans.date >= "2021-03-01"]
dates = chart_data["date"].unique()
for i,cr in countries.iterrows():
    c = cr["country"]
    text_disp_date = pd.to_datetime(cr["text_disp_date"]) if cr["text_disp_date"]!="" else dates.max()
    plt.plot_date(dates,chart_data.loc[chart_data["country"]==c]["inc_move_ave7_milpop"],label=c, fmt="-")
#    plt.text(dates.max(),chart_data.loc[(chart_data["country"]==c) & (chart_data["date"]==dates.max())]["inc_move_ave7_milpop"],c
    plt.text(text_disp_date,chart_data.loc[(chart_data["country"]==c) & (chart_data["date"]==text_disp_date)]["inc_move_ave7_milpop"],c
            ,fontsize=8,va="center")
plt.legend(bbox_to_anchor=(0, 1),loc='upper left', fontsize=9)

plt.subplot(2,1,2) # (rows, columns, panel number)
plt.grid()
plt.title("東・東南アジア主要国の人口100万人当たりの新規感染者数７日間移動平均({}時点)".format(latest_date_str))
chart_data = e_a_trans[e_a_trans.date >= "2021-03-01"]
dates = chart_data["date"].unique()
for i,cr in e_asia_c.iterrows():
    c = cr["country"]
    text_disp_date = pd.to_datetime(cr["text_disp_date"]) if cr["text_disp_date"]!="" else dates.max()
    plt.plot_date(dates,chart_data.loc[chart_data["country"]==c]["inc_move_ave7_milpop"],label=c, fmt="-")
#    plt.text(dates.max(),chart_data.loc[(chart_data["country"]==c) & (chart_data["date"]==dates.max())]["inc_move_ave7_milpop"],c
    plt.text(text_disp_date,chart_data.loc[(chart_data["country"]==c) & (chart_data["date"]==text_disp_date)]["inc_move_ave7_milpop"],c
             ,fontsize=8,va="center")
plt.legend(bbox_to_anchor=(0, 1),loc='upper left', fontsize=9)
plt.text(dates.max(),-1,"※ChinaはProvince/State毎の値を合算",ha="right",va="top",fontsize=8)

fig.text(0,0
         ,"※感染者の情報提供:COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University,"\
         +"※人口の情報提供:WHO",
        fontsize=7,ha="left",va="bottom")
fig.tight_layout()


<IPython.core.display.Javascript object>

/tmp/ipykernel_9414/1052152310.py:36: UserWarning: Glyph 20027 (\N{CJK UNIFIED IDEOGRAPH-4E3B}) missing from current font.
  fig.tight_layout()
/tmp/ipykernel_9414/1052152310.py:36: UserWarning: Glyph 35201 (\N{CJK UNIFIED IDEOGRAPH-8981}) missing from current font.
  fig.tight_layout()
/tmp/ipykernel_9414/1052152310.py:36: UserWarning: Glyph 22269 (\N{CJK UNIFIED IDEOGRAPH-56FD}) missing from current font.
  fig.tight_layout()
/tmp/ipykernel_9414/1052152310.py:36: UserWarning: Glyph 12398 (\N{HIRAGANA LETTER NO}) missing from current font.
  fig.tight_layout()
/tmp/ipykernel_9414/1052152310.py:36: UserWarning: Glyph 20154 (\N{CJK UNIFIED IDEOGRAPH-4EBA}) missing from current font.
  fig.tight_layout()
/tmp/ipykernel_9414/1052152310.py:36: UserWarning: Glyph 21475 (\N{CJK UNIFIED IDEOGRAPH-53E3}) missing from current font.
  fig.tight_layout()
/tmp/ipykernel_9414/1052152310.py:36: UserWarning: Glyph 19975 (\N{CJK UNIFIED IDEOGRAPH-4E07}) missing from current font.
  fig.tight_layout()


In [64]:
fig.savefig("out/covid-global-countries-transition.png")

/tmp/ipykernel_9414/346975147.py:1: UserWarning: Glyph 20027 (\N{CJK UNIFIED IDEOGRAPH-4E3B}) missing from current font.
  fig.savefig("out/covid-global-countries-transition.png")
/tmp/ipykernel_9414/346975147.py:1: UserWarning: Glyph 35201 (\N{CJK UNIFIED IDEOGRAPH-8981}) missing from current font.
  fig.savefig("out/covid-global-countries-transition.png")
/tmp/ipykernel_9414/346975147.py:1: UserWarning: Glyph 22269 (\N{CJK UNIFIED IDEOGRAPH-56FD}) missing from current font.
  fig.savefig("out/covid-global-countries-transition.png")
/tmp/ipykernel_9414/346975147.py:1: UserWarning: Glyph 12398 (\N{HIRAGANA LETTER NO}) missing from current font.
  fig.savefig("out/covid-global-countries-transition.png")
/tmp/ipykernel_9414/346975147.py:1: UserWarning: Glyph 20154 (\N{CJK UNIFIED IDEOGRAPH-4EBA}) missing from current font.
  fig.savefig("out/covid-global-countries-transition.png")
/tmp/ipykernel_9414/346975147.py:1: UserWarning: Glyph 21475 (\N{CJK UNIFIED IDEOGRAPH-53E3}) missing from c